## Ensemble

In [3]:
# Libraries
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier
import pickle
from pickle import load

root_path = "../../Data/GoogleDrive/"

In [4]:
# Load data
X_train = pd.read_parquet(root_path + "X_train.parquet")
X_test = pd.read_parquet(root_path + "X_test.parquet")
y_train = pd.read_parquet(root_path + "y_train.parquet")
y_test = pd.read_parquet(root_path + "y_test.parquet")

In [7]:
# Load models

# Root directory
root = '../../Data/GoogleDrive/'

# File names
log_name = 'logit_model_select.pkl'
knn_name = 'knn_model_full.pkl'
rf_name = 'rf_model_full.pkl'
xgb_name = 'xgboost_model_full.pkl'

with open(root + log_name, 'rb') as f:
    logit_model = pickle.load(f)

with open(root + knn_name, 'rb') as f:
    knn_model = pickle.load(f)

with open(root + rf_name, 'rb') as f:
    rf_model = pickle.load(f)

with open(root + xgb_name, 'rb') as f:
    xgb_model = pickle.load(f)

In [9]:
# Model
ensemble = VotingClassifier(
    estimators=[('logit', logit_model), 
                ('knn', knn_model),
                ('rf', rf_model),
                ('xgb', xgb_model)], 
                voting='soft')

ensemble.fit(X_train, y_train)
ensemble.score(X_test, y_test)
print('Ensemble model accuracy: ', ensemble.score(X_test, y_test))

# Predictions
y_pred = ensemble.predict(X_test)

/Users/hinnz/Downloads/anaconda3/envs/BIOSTAT203C-24S/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/hinnz/Downloads/anaconda3/envs/BIOSTAT203C-24S/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


KeyboardInterrupt: 

In [ ]:
# create confusion matrix
ensemble_cf = confusion_matrix(y_test, y_pred)
ensemble_cf = pd.DataFrame(ensemble_cf, 
                         columns = ['predicted_no_heart_disease', 
                                    'predicted_heart_disease'],
                         index = ['no_heart_disease',
                                  'has_heart_disease'])

In [ ]:
# Evaluate

# Accuracy
ens_accuracy = accuracy_score(y_test, y_pred)

# Precision
ens_precision = precision_score(y_test, y_pred)

# Recall
ens_recall = recall_score(y_test, y_pred)

# F1
ens_f1 = f1_score(y_test, y_pred)

# AUC ROC
ens_roc = roc_auc_score(y_test, y_pred)